In [11]:
from pyspark import SparkConf, SparkContext

In [2]:
import collections

In [3]:
conf = SparkConf().setMaster("local").setAppName("Practice")

In [4]:
sc = SparkContext(conf= conf)

# GENERATING OUR OWN JSON DATA

In [5]:
stringJSONRDD = sc.parallelize(("""
{ "id": "123",
"name": "Katie",
"age": 19,
"eyeColor": "brown"
}""",
"""{
"id": "234",
"name": "Michael",
"age": 22,
"eyeColor": "green"
}""",
"""{
"id": "345",
"name": "Simone",
"age": 23,
"eyeColor": "blue"
}""")
)

In [6]:
stringJSONRDD.take(1)

['\n{ "id": "123",\n"name": "Katie",\n"age": 19,\n"eyeColor": "brown"\n}']

# CONVERTING JSON RDD TO DATAFRAME

https://github.com/apache/spark/blob/master/examples/src/main/python/sql/basic.py

In [10]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [25]:
swimmersJSON = spark.read.json(stringJSONRDD)

In [26]:
#Running the.show() method will default to present the first 10 rows.
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [27]:
swimmersJSON.head(3)

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [29]:
swimmersJSON.select("age").show()

+---+
|age|
+---+
| 19|
| 22|
| 23|
+---+



In [28]:
swimmersJSON.select("name").show()

+-------+
|   name|
+-------+
|  Katie|
|Michael|
| Simone|
+-------+



In [31]:
swimmersJSON.filter(swimmersJSON['age'] > 21).show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [22]:
from pyspark.sql.types import *

In [35]:
sqlContext = SQLContext(spark)

In [ ]:
users.registerTempTable("users")
val data = sqlContext.sql("select username from users where username = 'tc'")

In [42]:
swimmersJSON.registerTempTable("swimmersJSON")
val data = sqlContext.sql("select * from swimmersJSON")

SyntaxError: invalid syntax (<ipython-input-42-f9573e8ba49e>, line 2)

if you are using Databricks, you can use the %sql command
and run your SQL statement directly within a notebook cell, as noted

%sql
Select * from swimmersJSON

In [43]:
#Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



# Programmatically specifying the schema

In [44]:

# Import types
from pyspark.sql.types import *


In [45]:
# Generate comma delimited data
stringCSVRDD = sc.parallelize([
(123, 'Katie', 19, 'brown'),
(234, 'Michael', 22, 'green'),
(345, 'Simone', 23, 'blue')
])

In [46]:
# Specify schema
schema = StructType([
StructField("id", LongType(), True),
StructField("name", StringType(), True),
StructField("age", LongType(), True),
StructField("eyeColor", StringType(), True)
])

In [47]:
# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [48]:
# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [49]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



# Querying with the DataFrame API

In [50]:
swimmers.count()

3

In [53]:
spark.sql("select count(*) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [54]:
# Get the id, age where age = 22 in SQL
spark.sql("select id,age from swimmers where age =22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [58]:
swimmers.select("id","age").filter("age=22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [59]:
# Another way to write the above query is below
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age
== 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [55]:
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [57]:
spark.sql("select name, eyeColor from swimmers where eyeColor like'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



# BIG EXAMPLE

# PREPARE DATA FOR MODELING

In [60]:
#Checking for duplicates, missing observations, and outliers

In [61]:
df = spark.createDataFrame([
(1, 144.5, 5.9, 33, 'M'),
(2, 167.2, 5.4, 45, 'M'),
(3, 124.1, 5.2, 23, 'F'),
(4, 144.5, 5.9, 33, 'M'),
(5, 133.2, 5.7, 54, 'F'),
(3, 124.1, 5.2, 23, 'F'),
(5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])

In [62]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [63]:
df.distinct().show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [64]:
print("Number of Distinct Rows:", df.distinct().count())

Number of Distinct Rows: 6


In [65]:
#We can drop these rows by using the .dropDuplicates(...) method:

In [66]:
df_dupFree = df.dropDuplicates()

In [67]:
df_dupFree.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [68]:
print('Count of ids: {0}'.format(df_dupFree.count()))

Count of ids: 6


In [70]:
df_dupFree.columns

['id', 'weight', 'height', 'age', 'gender']

In [71]:
df_dupFree.select([c for c in df_dupFree.columns if c != 'id']).distinct().show()

+------+------+---+------+
|weight|height|age|gender|
+------+------+---+------+
| 133.2|   5.7| 54|     F|
| 144.5|   5.9| 33|     M|
| 167.2|   5.4| 45|     M|
| 124.1|   5.2| 23|     F|
| 129.2|   5.3| 42|     M|
+------+------+---+------+



In [81]:
df_new = df.dropDuplicates(subset=[
c for c in df.columns if c != 'id'
])

In [82]:
df_new.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [74]:
#Checking for duplicate IDs

In [75]:
import pyspark.sql.functions as fn

In [84]:
df_new.agg(fn.countDistinct('id').alias('Count of Distinct ID')).show()

+--------------------+
|Count of Distinct ID|
+--------------------+
|                   4|
+--------------------+



In [85]:
df_new.withColumn('new_id',
fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



In [86]:
#New_ID column makes the dataframe unique now

# Missing observations

In [87]:
df_miss = spark.createDataFrame([ (1, 143.5, 5.6, 28,
'M', 100000),
(2, 167.2, 5.4, 45, 'M', None),
(3, None , 5.2, None, None, None),
(4, 144.5, 5.9, 33, 'M', None),
(5, 133.2, 5.7, 54, 'F', None),
(6, 124.1, 5.2, None, 'F', None),
(7, 129.2, 5.3, 42, 'M', 76000),
], ['id', 'weight', 'height', 'age', 'gender', 'income'])

In [88]:
df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [89]:
#To find the number of missing observations per row, we can use the following snippet:
df_miss.rdd.map(
lambda row: (row['id'], sum([c == None for c in row]))
).collect()
#It produces the following output:

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [92]:
#To find the PErcentage of missing observations per column, we can use the following snippet:
df_miss.agg(*[
(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
for c in df_miss.columns
]).show()
#It produces the following output:

+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



In [93]:
#To find the PErcentage of missing observations per column, we can use the following snippet:
df_miss.agg(*[
((1 - (fn.count(c) / fn.count('*')))*100).alias(c + '_missing')
for c in df_miss.columns
]).show()
#It produces the following output:

+----------+-----------------+--------------+-----------------+-----------------+-----------------+
|id_missing|   weight_missing|height_missing|      age_missing|   gender_missing|   income_missing|
+----------+-----------------+--------------+-----------------+-----------------+-----------------+
|       0.0|14.28571428571429|           0.0|28.57142857142857|14.28571428571429|71.42857142857143|
+----------+-----------------+--------------+-----------------+-----------------+-----------------+



In [94]:
#As most of the values in the INCOME column is missing, we shall remove it from the dataset
df_miss_no_income = df_miss.select([ c for c in df_miss.columns if c != 'income'])

In [96]:
df_miss_no_income.show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



However, if you decide to drop the observations instead, you can use the
.dropna(...) method, as shown here. Here, we will also use the thresh
parameter, which allows us to specify a threshold on the number of
missing observations per row that would qualify the row to be dropped.
This is useful if you have a dataset with tens or hundreds of features and
you only want to drop those rows that exceed a certain threshold of
missing values:

In [101]:
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [102]:
means = df_miss_no_income.agg(
*[fn.mean(c).alias(c)
for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]
means['gender'] = 'missing'


df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



# OUTLIER ANALYSIS

In [103]:
df_outliers = spark.createDataFrame([
(1, 143.5, 5.3, 28),
(2, 154.2, 5.5, 45),
(3, 342.3, 5.1, 99),
(4, 144.5, 5.5, 33),
(5, 133.2, 5.4, 54),
(6, 124.1, 5.1, 21),
(7, 129.2, 5.3, 42),
], ['id', 'weight', 'height', 'age'])

In [105]:
df_outliers.show()

+---+------+------+---+
| id|weight|height|age|
+---+------+------+---+
|  1| 143.5|   5.3| 28|
|  2| 154.2|   5.5| 45|
|  3| 342.3|   5.1| 99|
|  4| 144.5|   5.5| 33|
|  5| 133.2|   5.4| 54|
|  6| 124.1|   5.1| 21|
|  7| 129.2|   5.3| 42|
+---+------+------+---+



In [119]:
cols = ['weight', 'height', 'age']
bounds = {}
for col in cols:
    quantiles = df_outliers.approxQuantile(
    col, [0.25, 0.75], 0.05
    )
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
]

In [120]:
bounds

{'age': [-11.0, 93.0],
 'height': [4.499999999999999, 6.1000000000000005],
 'weight': [91.69999999999999, 191.7]}

In [121]:
#Identifying the outliers

In [122]:
outliers = df_outliers.select(*['id'] + [
(
(df_outliers[c] < bounds[c][0]) |
(df_outliers[c] > bounds[c][1])
).alias(c + '_o') for c in cols

])
outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+



In [123]:
df_outliers = df_outliers.join(outliers, on='id')


In [124]:
df_outliers.show()

+---+------+------+---+--------+--------+-----+
| id|weight|height|age|weight_o|height_o|age_o|
+---+------+------+---+--------+--------+-----+
|  7| 129.2|   5.3| 42|   false|   false|false|
|  6| 124.1|   5.1| 21|   false|   false|false|
|  5| 133.2|   5.4| 54|   false|   false|false|
|  1| 143.5|   5.3| 28|   false|   false|false|
|  3| 342.3|   5.1| 99|    true|   false| true|
|  2| 154.2|   5.5| 45|   false|   false|false|
|  4| 144.5|   5.5| 33|   false|   false|false|
+---+------+------+---+--------+--------+-----+



In [125]:
df_outliers.filter('weight_o').select('id', 'weight').show()

+---+------+
| id|weight|
+---+------+
|  3| 342.3|
+---+------+



In [287]:
df_outliers.filter('age_o').select('id', 'age').show()

Py4JJavaError: An error occurred while calling o4264.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 293.0 failed 1 times, most recent failure: Lost task 0.0 in stage 293.0 (TID 3793, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\23\temp_shuffle_ba1fb7c3-cff8-4adb-b615-d47fe524811b (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(Bypass

# DESCRIPTIVE STATISTICS --FRAUD RISK DATASET

In [244]:
import pyspark.sql.types as typ

In [245]:
#Reading the data

fraud = sc.textFile('ccFraud.csv')

In [246]:
#Checking the columns
header = fraud.first()

In [247]:
header

'"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

In [277]:
fraud.count()

10000000

In [248]:
fraud.take(2)

['"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"',
 '1,1,35,1,3000,4,14,2,0']

In [249]:
#As it is a comma separated value, we shall be splitting with separater as a comma(,)

In [250]:
fraud = fraud \
.filter(lambda row: row != header) \
.map(lambda row: [int(elem) for elem in row.split(',')])

In [251]:
#Next, we create the schema for our DataFrame:

In [252]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True)
        for h in header.split(',')
    ]
] 
schema = typ.StructType(fields)

In [253]:
fields

[StructField(custID,IntegerType,true),
 StructField(gender,IntegerType,true),
 StructField(state,IntegerType,true),
 StructField(cardholder,IntegerType,true),
 StructField(balance,IntegerType,true),
 StructField(numTrans,IntegerType,true),
 StructField(numIntlTrans,IntegerType,true),
 StructField(creditLine,IntegerType,true),
 StructField(fraudRisk,IntegerType,true)]

In [254]:
#Finally, we create our DataFrame

In [255]:
fraud_df = spark.createDataFrame(fraud, schema)

In [256]:
fraud_df.printSchema()

root
 |-- custID: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- state: integer (nullable = true)
 |-- cardholder: integer (nullable = true)
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)
 |-- creditLine: integer (nullable = true)
 |-- fraudRisk: integer (nullable = true)



In [257]:
fraud_df.head()

Row(custID=1, gender=1, state=35, cardholder=1, balance=3000, numTrans=4, numIntlTrans=14, creditLine=2, fraudRisk=0)

In [258]:
fraud_df.show(5)

+------+------+-----+----------+-------+--------+------------+----------+---------+
|custID|gender|state|cardholder|balance|numTrans|numIntlTrans|creditLine|fraudRisk|
+------+------+-----+----------+-------+--------+------------+----------+---------+
|     1|     1|   35|         1|   3000|       4|          14|         2|        0|
|     2|     2|    2|         1|      0|       9|           0|        18|        0|
|     3|     2|    2|         1|      0|      27|           9|        16|        0|
|     4|     1|   15|         1|      0|      12|           0|         5|        0|
|     5|     1|   46|         1|      0|      11|          16|         7|        0|
+------+------+-----+----------+-------+--------+------------+----------+---------+
only showing top 5 rows



In [280]:
fraud_df.columns

['custID',
 'gender',
 'state',
 'cardholder',
 'balance',
 'numTrans',
 'numIntlTrans',
 'creditLine',
 'fraudRisk']

fraud_df.na.replace ('', 'Unknown')

In [266]:
from pyspark.sql.functions import col, when
def blank_as_null(x):
    return when(col(x) != '', col(x))

In [267]:
to_convert = ['custID',
 'gender',
 'state',
 'cardholder',
 'balance',
 'numTrans',
 'numIntlTrans',
 'creditLine',
 'fraudRisk']

exprs = [
    blank_as_null(x).alias(x) if x in to_convert else x for x in fraud_df.columns]

fraud_df.select(*exprs)

DataFrame[custID: int, gender: int, state: int, cardholder: int, balance: int, numTrans: int, numIntlTrans: int, creditLine: int, fraudRisk: int]

In [268]:
display(fraud_df.filter("state IS NOT NULL").count())

Py4JJavaError: An error occurred while calling o3831.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 274.0 failed 1 times, most recent failure: Lost task 0.0 in stage 274.0 (TID 3767, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\1b\temp_shuffle_950b08a2-baf2-417d-8913-83b2c357693d (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMerge

In [214]:
fraud_df.where(col("gender").isNull()).count().show()

Py4JJavaError: An error occurred while calling o3029.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 248.0 failed 1 times, most recent failure: Lost task 0.0 in stage 248.0 (TID 3740, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\2c\shuffle_48_0_0.data.2edb37d2-036e-4bf0-9f65-25ef8365bc30 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.writePartitionedFile(BypassMergeSortShuffleWriter.java:191)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:163)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.Shuffl

fraud_df.groupBy("gender").count().show()

In [288]:
#To find the PErcentage of missing observations per column, we can use the following snippet:
import pyspark.sql.functions as fn

fraud_df.agg(*[
(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
for c in fraud_df.columns
]).show()
#It produces the following output:

Py4JJavaError: An error occurred while calling o4365.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 296.0 failed 1 times, most recent failure: Lost task 0.0 in stage 296.0 (TID 3795, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\3e\temp_shuffle_963d2999-06c2-4f87-aec7-1a33f10ec3a5 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(Bypass


fraud_df.filter(

In [227]:
fraud_df.columns

['custID',
 'gender',
 'state',
 'cardholder',
 'balance',
 'numTrans',
 'numIntlTrans',
 'creditLine',
 'fraudRisk']

In [236]:
fraud_df.describe()

Py4JJavaError: An error occurred while calling o2659.describe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 258.0 failed 1 times, most recent failure: Lost task 0.0 in stage 258.0 (TID 3751, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\21\temp_shuffle_77976eb1-5da0-4ebc-b53e-6daa002d58df (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMe

In [274]:
fraud_df_na = fraud_df.dropna()

In [276]:
fraud_df.count()

Py4JJavaError: An error occurred while calling o3435.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 284.0 failed 1 times, most recent failure: Lost task 0.0 in stage 284.0 (TID 3777, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\39\temp_shuffle_e10eee5a-6007-45e5-a0c5-3d64690fc698 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMerge

In [282]:
fraud_df_na= fraud_df.na.fill(-1)

In [283]:
fraud_df_na.groupBy('gender').count().show()

Py4JJavaError: An error occurred while calling o4150.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 288.0 failed 1 times, most recent failure: Lost task 0.0 in stage 288.0 (TID 3789, localhost, executor driver): java.io.FileNotFoundException: C:\Users\adi19\AppData\Local\Temp\blockmgr-5da5edd5-a790-4623-a864-e027bd7a9878\09\temp_shuffle_0d92213d-479a-46cb-8352-8b18e7068c09 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(Bypass

In [285]:
fraud_df[fraud_df['gender'].apply(lambda x: type(x) == str)]

TypeError: 'Column' object is not callable